In [ ]:
from exptools import *

In [ ]:
from datetime import datetime, timezone
from pathlib import Path


SEED = b'\x81\x97u+'

MODEL_NAME = 'experimental_model_ivan 2'
MODEL_PATH = Path(f"{MODEL_NAME}.decl")

EXPORT_PATH = Path('output', 'tests_' + datetime.now(timezone.utc).isoformat(timespec='minutes'))
EXPORT_PATH.mkdir(parents=True, exist_ok=False)

In [ ]:
from Declare4Py.ProcessMiningTasks.LogGenerator.PositionalBased.PositionalBasedLogGeneratorNG import PBLogGeneratorHamming


traces = 10
events = 20
noise = 0
model = PositionalBasedModel().parse_from_file(MODEL_PATH.as_posix())

exp = Experiment(
        id_= 'random',
        class_=PBLogGeneratorHamming,
        model=model,
        parameters={
            'traces': traces,
            'events': events,
        },
        args={
            'init': {
                'total_traces': traces,
                'min_event': events,
                'max_event': events,
                'process_model': model,
                'log': None,
                'verbose': True,
                'seed': SEED,
                'threshold': .3,
                'randomise': True},
            'run': {
                'equal_rule_split': True,
                'high_variability': False,
                'generate_negatives_traces': False,
                'positive_noise_percentage': noise,
                'negative_noise_percentage': noise,
                'append_results': False}}
                )
g: PBLogGeneratorHamming = exp.run_generator()

exp.get_results(g)

In [ ]:
print(g._tabu_asp_program())

## Testing reproducibility

In [ ]:
from Declare4Py.ProcessMiningTasks.LogGenerator.PositionalBased.PositionalBasedLogGeneratorNG import PBLogGeneratorHamming, PBLogGeneratorRandom


traces = 10
events = 20
noise = 0
model = PositionalBasedModel().parse_from_file(MODEL_PATH.as_posix())

REPRODUCIBILITY_EXP = {
    'orig': Experiment(
        id_= 'orig',
        class_=PositionalBasedLogGenerator,
        model=model,
        args={
            'init': {
                'total_traces': traces,
                'min_events': events,
                'max_events': events,
                'pb_model': model,
                'verbose': False},
            'run': {
                'equal_rule_split': True,
                'high_variability': False,
                'generate_negatives_traces': False,
                'positive_noise_percentage': noise,
                'negative_noise_percentage': noise,
                'append_results': False,}}
                ),
    'random': Experiment(
        id_= 'random',
        class_=PBLogGeneratorRandom,
        model=model,
        args={
            'init': {
                'total_traces': traces,
                'min_event': events,
                'max_event': events,
                'process_model': model,
                'log': None,
                'verbose': False,
                'seed': SEED},
            'run': {
                'equal_rule_split': True,
                'high_variability': False,
                'generate_negatives_traces': False,
                'positive_noise_percentage': noise,
                'negative_noise_percentage': noise,
                'append_results': False}}
                ),
    'hamming': Experiment(
        id_= 'random',
        class_=PBLogGeneratorHamming,
        model=model,
        args={
            'init': {
                'total_traces': traces,
                'min_event': events,
                'max_event': events,
                'process_model': model,
                'log': None,
                'verbose': False,
                'seed': SEED,
                'threshold': .8,
                'randomise': True},
            'run': {
                'equal_rule_split': True,
                'high_variability': False,
                'generate_negatives_traces': False,
                'positive_noise_percentage': noise,
                'negative_noise_percentage': noise,
                'append_results': False}}
                )
}

REPRODUCIBILITY_EXP

In [ ]:
for exp, seed in itertools.product(REPRODUCIBILITY_EXP.values(), (None, SEED)):
    header = '-' * 5 + f' checking reproducibility of {exp.id_} with seed={seed} '
    print(header + '-' * (72 - len(header)))
    diff = exp.check_reproducibility(seed=seed)
    print(diff)
